# TSA Chapter 2: ARMA(1,1) Process

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch2/TSA_ch2_arma/TSA_ch2_arma.ipynb)

This notebook demonstrates:
- ARMA(p,q) combined AR and MA components
- Simulation of ARMA(1,1) with different parameter combinations
- Both ACF and PACF decay without sharp cutoff

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Set random seed

n = 500

print("=" * 60)
print("ARMA(p,q) PROCESS")
print("=" * 60)

print("""
ARMA(p,q) Model:
  X_t = c + φ₁X_{t-1} + ... + φₚX_{t-p} + ε_t + θ₁ε_{t-1} + ... + θqε_{t-q}

Lag Notation:
  φ(L)X_t = θ(L)ε_t

Stationarity: Roots of φ(z) = 0 outside unit circle
Invertibility: Roots of θ(z) = 0 outside unit circle

Special Cases:
  ARMA(p,0) = AR(p)
  ARMA(0,q) = MA(q)
""")

# Simulate ARMA(1,1) with different parameters
params = [
    (0.7, 0.4, 'ARMA(1,1): φ=0.7, θ=0.4'),
    (0.7, -0.4, 'ARMA(1,1): φ=0.7, θ=-0.4'),
    (-0.5, 0.6, 'ARMA(1,1): φ=-0.5, θ=0.6'),
    (0.9, 0.3, 'ARMA(1,1): φ=0.9, θ=0.3 (persistent)')
]

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for i, (phi, theta, title) in enumerate(params):
    # Create ARMA process
    ar = np.array([1, -phi])  # AR coefficients: 1 - φL
    ma = np.array([1, theta])  # MA coefficients: 1 + θL
    arma_process = ArmaProcess(ar, ma)

    # Simulate
    x = arma_process.generate_sample(nsample=n)

    # Time series plot
    axes[0, i].plot(x[:200], 'b-', linewidth=0.8, alpha=0.8)
    axes[0, i].axhline(y=0, color='red', linestyle='--', linewidth=1)
    axes[0, i].set_title(title, fontsize=10)
    axes[0, i].set_xlabel('Time')
    axes[0, i].set_ylabel('X_t')
    axes[0, i].grid(True, alpha=0.3)

    # ACF plot
    acf_values = acf(x, nlags=15)
    axes[1, i].bar(range(16), acf_values, color='blue', alpha=0.7)
    axes[1, i].axhline(y=1.96/np.sqrt(n), color='red', linestyle='--')
    axes[1, i].axhline(y=-1.96/np.sqrt(n), color='red', linestyle='--')
    axes[1, i].set_title('ACF: decays (mixed pattern)', fontsize=10)
    axes[1, i].set_xlabel('Lag')
    axes[1, i].set_ylabel('ACF')
    axes[1, i].grid(True, alpha=0.3)

    # PACF plot
    pacf_values = pacf(x, nlags=15)
    axes[2, i].bar(range(16), pacf_values, color='green', alpha=0.7)
    axes[2, i].axhline(y=1.96/np.sqrt(n), color='red', linestyle='--')
    axes[2, i].axhline(y=-1.96/np.sqrt(n), color='red', linestyle='--')
    axes[2, i].set_title('PACF: decays (mixed pattern)', fontsize=10)
    axes[2, i].set_xlabel('Lag')
    axes[2, i].set_ylabel('PACF')
    axes[2, i].grid(True, alpha=0.3)

plt.tight_layout()
save_chart(fig, 'ch2_arma_properties')
plt.show()

# Model identification guide
print("\n" + "=" * 60)
print("MODEL IDENTIFICATION FROM ACF/PACF PATTERNS")
print("=" * 60)
print("""
┌─────────────┬──────────────────────┬──────────────────────┐
│   Model     │        ACF           │        PACF          │
├─────────────┼──────────────────────┼──────────────────────┤
│   AR(p)     │  Decays (exp/osc)    │  Cuts off at lag p   │
│   MA(q)     │  Cuts off at lag q   │  Decays (exp/osc)    │
│  ARMA(p,q)  │  Decays after lag q  │  Decays after lag p  │
└─────────────┴──────────────────────┴──────────────────────┘

ARMA Identification is HARD:
  - Both ACF and PACF decay
  - No clean cutoff
  - Use information criteria (AIC, BIC) to compare models
""")

# ARMA(1,1) formulas
print("\n" + "=" * 60)
print("ARMA(1,1) FORMULAS")
print("=" * 60)

phi, theta = 0.7, 0.4
sigma_sq = 1

print(f"""
Model: X_t = φX_{{t-1}} + ε_t + θε_{{t-1}}
       X_t = {phi}X_{{t-1}} + ε_t + {theta}ε_{{t-1}}

Mean: E[X_t] = 0 (assuming c = 0)

Variance:
  γ(0) = σ² × (1 + θ² + 2φθ) / (1 - φ²)
       = {sigma_sq} × (1 + {theta**2} + 2×{phi}×{theta}) / (1 - {phi**2})
       = {sigma_sq * (1 + theta**2 + 2*phi*theta) / (1 - phi**2):.4f}

ACF at lag 1:
  ρ(1) = (φ + θ)(1 + φθ) / (1 + θ² + 2φθ)
       = ({phi} + {theta})(1 + {phi}×{theta}) / (1 + {theta**2} + 2×{phi}×{theta})
       = {(phi + theta)*(1 + phi*theta) / (1 + theta**2 + 2*phi*theta):.4f}

ACF at lag h > 1:
  ρ(h) = φ × ρ(h-1)  (decays like AR)
""")